# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use Pandas to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Import used libs

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from __future__ import division

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = pd.read_csv('data/kc_house_data.csv')

# Split data into training and testing.

In [3]:
train_data = pd.read_csv('data/kc_house_train_data.csv')
test_data = pd.read_csv('data/kc_house_test_data.csv')

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

In [4]:
# Create linear regression object
regr = linear_model.LinearRegression()

input_features = ['sqft_living', 'bedrooms', 'bathrooms']

X_train = train_data[input_features]
y_train = train_data['price']

# Train the model using the training sets
regr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

Now that we have fitted the model we can extract the regression weights (coefficients)

In [5]:
print regr.intercept_
print regr.coef_

87912.865815
[   315.40669062 -65081.88711588   6942.16598637]


# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [6]:
example_predictions = regr.predict(train_data[input_features])
print example_predictions[0] # should be 271789.505878

271789.26538


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [7]:
def get_residual_sum_of_squares(model, data, outcome):
    # Compute the predictions, the residuals/errors, and then square and add them up
    RSS = np.sum(((model.predict(data) - outcome) ** 2))

    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [8]:
rss_example_train = get_residual_sum_of_squares(regr, test_data[input_features], test_data['price'])
print rss_example_train # should be 2.7376153833e+14

2.73761940583e+14


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [9]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [10]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [11]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['bed_bath_rooms'] = train_data[['bedrooms','bathrooms']].apply(lambda row: row['bedrooms']*row['bathrooms'], axis=1)
test_data['bed_bath_rooms'] = test_data[['bedrooms','bathrooms']].apply(lambda row: row['bedrooms']*row['bathrooms'], axis=1)
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))
train_data['lat_plus_long'] = train_data[['lat','long']].apply(lambda row: row['lat']+row['long'], axis=1)
test_data['lat_plus_long'] = test_data[['lat','long']].apply(lambda row: row['lat']+row['long'], axis=1)

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [12]:
print "bedrooms_squared = %0.2f" % (test_data['bedrooms_squared'].mean())
print "bed_bath_rooms = %0.2f" % (test_data['bed_bath_rooms'].mean())
print "log_sqft_living = %0.2f" % (test_data['log_sqft_living'].mean())
print "lat_plus_long = %0.2f" % (test_data['lat_plus_long'].mean())

bedrooms_squared = 12.45
bed_bath_rooms = 7.50
log_sqft_living = 7.55
lat_plus_long = -74.65


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [13]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [14]:
# Learn the three models:
regr_1 = linear_model.LinearRegression()
regr_2 = linear_model.LinearRegression()
regr_3 = linear_model.LinearRegression()

X_train_1 = train_data[model_1_features]
X_train_2 = train_data[model_2_features]
X_train_3 = train_data[model_3_features]
y_train = train_data['price']

regr_1.fit(X_train_1, y_train)
regr_2.fit(X_train_2, y_train)
regr_3.fit(X_train_3, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [15]:
# Examine/extract each model's coefficients:
print regr_1.coef_
print regr_2.coef_
print regr_3.coef_

[  3.12258646e+02  -5.95865332e+04   1.57067421e+04   6.58619264e+05
  -3.09374351e+05]
[  3.06610053e+02  -1.13446368e+05  -7.14613083e+04   6.54844630e+05
  -2.94298969e+05   2.55796520e+04]
[  5.29422820e+02   3.45142296e+04   6.70607813e+04   5.34085611e+05
  -4.06750711e+05  -8.57050439e+03  -6.78858667e+03  -5.61831484e+05
   1.27334900e+05]


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [16]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
rss_train_1 = get_residual_sum_of_squares(regr_1, train_data[model_1_features], train_data['price'])
print 'RSS model 1: ' + rss_train_1.astype('str')
rss_train_2 = get_residual_sum_of_squares(regr_2, train_data[model_2_features], train_data['price'])
print 'RSS model 2: ' + rss_train_2.astype('str')
rss_train_3 = get_residual_sum_of_squares(regr_3, train_data[model_3_features], train_data['price'])
print 'RSS model 3: ' + rss_train_3.astype('str')

RSS model 1: 9.6787996305e+14
RSS model 2: 9.58419635074e+14
RSS model 3: 9.0343645505e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [17]:
# Compute the RSS on TESTING data for each of the three models and record the values:
rss_test_1 = get_residual_sum_of_squares(regr_1, test_data[model_1_features], test_data['price'])
print 'RSS model 1: ' + rss_test_1.astype('str')
rss_test_2 = get_residual_sum_of_squares(regr_2, test_data[model_2_features], test_data['price'])
print 'RSS model 2: ' + rss_test_2.astype('str')
rss_test_3 = get_residual_sum_of_squares(regr_3, test_data[model_3_features], test_data['price'])
print 'RSS model 3: ' + rss_test_3.astype('str')

RSS model 1: 2.25500469795e+14
RSS model 2: 2.23377462976e+14
RSS model 3: 2.59236319207e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.